# ICL data science challenge

## EDA

In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
train_path = "../input/raw/train.csv"
preprocessed_path = "../input/derived/train-preprocessed.csv"

In [11]:
df = pd.read_csv(preprocessed_path, sep=r'\s+', quotechar='"')
df

,Index,Artist,Track,Album,danceability,energy,loudness,speechiness,acousticness,instrumentalness,...,AnC,AnD,BnC,BnD,CnD,AnBnC,AnBnD,AnCnD,BnCnD,AnBnCnD
0,1,tajima hal,Grape Choice,Grape Choice,0.704,0.5870,-9.815,0.3120,0.341,0.004510,...,0,0,0,0,0,0,0,0,0,0
1,2,CHROMANCE,Wrap Me In Plastic - Marcus Layton Radio Edit,Wrap Me In Plastic (Marcus Layton Radio Edit),0.917,0.6120,-2.183,0.0854,0.238,0.000023,...,0,0,0,0,0,0,0,0,0,0
2,3,Stevie Wonder,Sir Duke,Songs In The Key Of Life,0.588,0.5680,-9.766,0.0672,0.160,0.000000,...,0,0,0,0,0,0,0,0,0,0
3,4,MishCatt,Midnight Sun - Acoustic Version,Midnight Sun (Acoustic Version),0.590,0.0991,-15.925,0.0490,0.915,0.000005,...,0,0,0,0,0,0,0,0,0,0
4,5,Gloria Estefan,Sólo Por Tu Amor,Alma Caribeña,0.561,0.4770,-8.123,0.0355,0.295,0.000000,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7588,7589,Some Sprouts,Someone You Love,Divinely Uninspired To A Hellish Extent,0.501,0.4050,-5.679,0.0319,0.751,0.000000,...,0,0,0,0,0,0,0,0,0,0
7589,7590,Sure Sure,Funky Galileo,Funky Galileo,0.802,0.3250,-9.327,0.0466,0.164,0.162000,...,0,0,0,0,0,0,0,0,0,0
7590,7591,The Flying Stars Of Brooklyn NY,Live On,My God Has a Telephone,0.513,0.6260,-8.843,0.0582,0.579,0.000633,...,0,0,0,0,0,0,0,0,0,0
7591,7592,Gasper Lawal,Kita Kita,Nigeria 70 - Funky Lagos,0.614,0.9070,-6.565,0.0472,0.062,0.772000,...,0,0,0,0,0,0,0,0,0,0


## Sanity checking cells

In [9]:
temp = ['AnB', 'AnC', 'AnD', 'BnC', 'BnD', 'CnD',
        'AnBnC', 'AnBnD', 'AnCnD', 'BnCnD', 'AnBnCnD']
for i in range(len(df['A'])):
    break
    for c in temp:
        row = df.iloc[i]
        if row[c] == 1:
            if 'A' in c:
                if row['A'] == 0:
                    print(row['Index'], 'A')
            if 'B' in c:
                if row['B'] == 0:
                    print(row['Index'], 'B')
            if 'C' in c:
                if row['C'] == 0:
                    print(row['Index'], 'C')
            if 'D' in c:
                if row['D'] == 0:
                    print(row['Index'], 'D')

234 A
330 B
752 A
2016 B
2261 A
2419 A
3531 B
3863 B
3873 B
3873 B
4375 A
4997 B
5230 B
5466 B
6337 A
6362 A
6627 B
7372 B
7432 A


In [10]:
df.columns

Index(['Index', 'Artist', 'Track', 'Album', 'danceability', 'energy',
       'loudness', 'speechiness', 'acousticness', 'instrumentalness',
       'liveness', 'tempo', 'length', 'time_signature', 'A', 'B', 'C', 'D',
       'AnB', 'AnC', 'AnD', 'BnC', 'BnD', 'CnD', 'AnBnC', 'AnBnD', 'AnCnD',
       'BnCnD', 'AnBnCnD'],
      dtype='object')

## Baseline model logistic regression

In [12]:
df.columns
feature_cols = ['danceability', 'energy',
       'loudness', 'speechiness', 'acousticness', 'instrumentalness',
       'liveness', 'tempo', 'length']
target_cols = ['A', 'B', 'C', 'D']

X = df[feature_cols].to_numpy()
y = df[target_cols].to_numpy()

In [13]:
X.shape, y.shape

((7593, 9), (7593, 4))

In [13]:
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

logRegs = [LogisticRegression(), LogisticRegression(), LogisticRegression(), LogisticRegression()]
accs = []

for i in range(4):
    logRegs[i].fit(X_train, y_train[:, i])
    preds = logRegs[i].predict(X_test)
    acc = accuracy_score(y_test[:, i], preds)
    accs.append(acc)
    print(f"Validation accuracy for class {target_cols[i]}: {acc:.4f}")
print(f"Average validation accuracy: {np.mean(accs)}")

Validation accuracy for class A: 0.6504
Validation accuracy for class B: 0.6274
Validation accuracy for class C: 0.9835
Validation accuracy for class D: 0.8308
Average validation accuracy: 0.7730414746543779


In [17]:
from src.preprocessing.data_formatting import get_preprocessed_data
# produce predictions on test data
df_test = get_preprocessed_data("../input/", skip_preprocess=False, get_test=True)

X_test = df_test[feature_cols].to_numpy()
X_test = scaler.transform(X_test)
for i in range(4):
    preds = logRegs[i].predict(X_test)
    df_test[target_cols[i]] = preds

array([[ 6.25000e-01,  7.94000e-01, -6.16800e+00, ...,  3.38000e-01,
         9.80200e+01,  1.94853e+05],
       [ 3.93000e-01,  5.88000e-01, -1.52320e+01, ...,  2.38000e-01,
         1.36535e+02,  2.04893e+05],
       [ 5.81000e-01,  7.81000e-01, -8.89000e+00, ...,  7.86000e-02,
         8.77570e+01,  1.81773e+05],
       ...,
       [ 6.37000e-01,  5.05000e-01, -7.33200e+00, ...,  1.10000e-01,
         8.03830e+01,  1.55773e+05],
       [ 4.34000e-01,  9.39000e-01, -2.98300e+00, ...,  4.91000e-01,
         1.59932e+02,  2.09493e+05],
       [ 6.34000e-01,  6.62000e-01, -7.54300e+00, ...,  1.11000e-01,
         9.00110e+01,  2.10933e+05]])

In [15]:
df_test['A'].mean()

1.0

In [74]:
def fill_remaining_cols(df):
    extended_cols = ['AnB', 'AnC', 'AnD', 'BnC', 'BnD', 'CnD', 'AnBnC', 'AnBnD', 'AnCnD', 'BnCnD', 'AnBnCnD']
    for i in range(len(df['A'])):
        for c in extended_cols:
            fill_val = True
            for l in ['A', 'B', 'C', 'D']:
                if l in c:
                    fill_val = fill_val and (df.at[i, l] == 1)
            # modify the cell
            df.at[i, c] = int(fill_val)

def submit_dataframe(df, submit_path):
    df = df.reset_index()
    assert len(df.columns) == 29
    df_test.to_csv(submit_path, index=False, quotechar='"', sep=' ')

submit_dataframe(df_test, "../output/2sigma.csv")

In [63]:
len(df_test.columns)

28

## Get more features

In [3]:
from src.preprocessing.data_formatting import get_preprocessed_data
import pandas as pd
import numpy as np
import re

df_train = get_preprocessed_data("../input", skip_preprocess=False, filename="train")
df_test = pd.read_csv("../input/raw/testanswer.csv")

del df_train['Index']
df_all = pd.concat([df_train, df_test], ignore_index=True, sort=False)

In [14]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import yaml
from tqdm.auto import tqdm
import time

with open("../config.yaml") as f:
     secrets = yaml.load(f, Loader=yaml.FullLoader)

# Authenticate with Spotify
client_id = secrets['clientID']
client_secret = secrets['clientSecret']

client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

def get_genre(sp, artist, track, album):
    # Search for the track using the given details
    query = f"artist:{artist} track:{track} album:{album}"
    results = sp.search(query)

    # Check if results were found
    if results['tracks']['items']:
        try:
            track_item = results['tracks']['items'][0] 
            # Get the main artist's ID from the first search result
            artist_id = track_item['artists'][0]['id']

            track_popularity = track_item['popularity']
            
            time.sleep(0.1)
            # Fetch artist details to get the genre
            artist_info = sp.artist(artist_id)
            artist_popularity = artist_info['popularity']

            return artist_info['genres'], track_popularity, artist_popularity
        except TypeError:
            return None, None, None
    else:
        return None, None, None

def get_genre_features(df, sp):
    genre_vocab = {}
    
    genre_list = []
    track_pop_list = []
    artist_pop_list = []
    
    if get_genre_features.cache is None:
        for i in tqdm(range(len(df))):
            row = df.iloc[i]
            time.sleep(0.1)
            genres, tpop, apop = get_genre(sp, row['Artist'], row['Track'], row['Album'])
            genre_list.append(genres)
            track_pop_list.append(tpop)
            artist_pop_list.append(apop)

        get_genre_features.cache = {
            'genres' : genre_list,
            'tracks' : track_pop_list,
            'artists' : artist_pop_list,
        }
    else:
        genre_list = get_genre_features.cache['genres']
        track_pop_list = get_genre_features.cache['tracks']
        artist_pop_list = get_genre_features.cache['tracks']
        
    return genre_list, track_pop_list, artist_pop_list
get_genre_features.cache = None

In [ ]:
temp_cache = get_genre_features(df_all, sp)

  0%|          | 0/8542 [00:00<?, ?it/s]

In [ ]:
temp_cache

In [ ]:
genre_list = temp_cache[0]
genre_vocab = {}

for row in genre_list:
    for item in row:
        words = [g for g in item.split(' ')]
        f

## Trying different models

In [13]:
from src.evaluation.metrics import competition_metric
from src.preprocessing.data_formatting import get_preprocessed_data
import pandas as pd
import numpy as np
import re

df_train = get_preprocessed_data("../input", skip_preprocess=False, filename="train")
df_test = pd.read_csv("../input/raw/testanswer.csv")

del df_train['Index']
df_all = pd.concat([df_train, df_test], ignore_index=True, sort=False)

df_all

,Artist,Track,Album,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,...,AnC,AnD,BnC,BnD,CnD,AnBnC,AnBnD,AnCnD,BnCnD,AnBnCnD
0,tajima hal,Grape Choice,Grape Choice,0.704,0.5870,-9.815,0.3120,0.341000,0.004510,0.1410,...,0,0,0,0,0,0,0,0,0,0
1,CHROMANCE,Wrap Me In Plastic - Marcus Layton Radio Edit,Wrap Me In Plastic (Marcus Layton Radio Edit),0.917,0.6120,-2.183,0.0854,0.238000,0.000023,0.0585,...,0,0,0,0,0,0,0,0,0,0
2,Stevie Wonder,Sir Duke,Songs In The Key Of Life,0.588,0.5680,-9.766,0.0672,0.160000,0.000000,0.0792,...,0,0,0,0,0,0,0,0,0,0
3,MishCatt,Midnight Sun - Acoustic Version,Midnight Sun (Acoustic Version),0.590,0.0991,-15.925,0.0490,0.915000,0.000005,0.1090,...,0,0,0,0,0,0,0,0,0,0
4,Gloria Estefan,Sólo Por Tu Amor,Alma Caribeña,0.561,0.4770,-8.123,0.0355,0.295000,0.000000,0.0778,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8537,The 1975,Lostmyhead,"I like it when you sleep, for you are so beaut...",0.341,0.6710,-7.481,0.0391,0.241000,0.863000,0.1120,...,0,0,0,0,0,0,0,0,0,0
8538,Klô Pelgag,Insomnie,Brol La Suite,0.496,0.3390,-10.512,0.0426,0.650000,0.000049,0.0945,...,0,0,0,0,0,0,0,0,0,0
8539,Barenaked Ladies,Smile - Acoustic,Grinning Streak (Deluxe Version),0.637,0.5050,-7.332,0.0406,0.268000,0.000007,0.1100,...,0,0,0,0,0,0,0,0,0,0
8540,Skillet,Awake and Alive,Awake,0.434,0.9390,-2.983,0.0593,0.042500,0.000000,0.4910,...,0,0,0,0,0,0,0,0,0,0


In [21]:
feature_cols = ['danceability', 'energy',
       'loudness', 'speechiness', 'acousticness', 'instrumentalness',
       'liveness', 'tempo', 'length']
target_cols = ['A', 'B', 'C', 'D']

# X = df[feature_cols].to_numpy()
# y = df[target_cols].to_numpy()
def get_one_hot(targets, nb_classes):
    res = np.eye(nb_classes)[np.array(targets).reshape(-1)]
    return res.reshape(list(targets.shape)+[nb_classes])

df_all[feature_cols]
time_sig_feature = np.zeros(
df_all['time_signature']

array([4, 1, 3, 5, 0])